In [5]:
import re
from pypdf import PdfReader, PdfWriter
from tqdm import tqdm

pattern1 = re.compile(r'^[0-9](?:\.[0-9]+)+ [A-Z].*', re.M)
pattern2 = re.compile(r'Contents|CONTENTS')

reader = PdfReader('Richard O. Duda, Peter E. Hart, David G. Stork - Pattern Classification (Wiley) - libgen.li.pdf')
writer = PdfWriter()

In [6]:
head_to_page = {}

for page_number, page in enumerate(tqdm(reader.pages)):
    if page_number < 12:
        continue
    text = page.extract_text()
    if pattern2.search(text) != None:
        if pattern2.search(text).group() == 'Contents':
            temp = text.split('\n')[1][:-1]
            if temp[0] != 'A':
                head_to_page[temp] = page_number
    else:
        for i in pattern1.findall(text):
            head_to_page[i] = page_number
    writer.add_page(page)

100%|██████████| 738/738 [00:25<00:00, 28.66it/s]


In [9]:
key_to_head = {}

def gen_key(head:str):
    temp = head.split()
    key = temp[0].split('.')
    key = [int(i) for i in key]
    while len(key) < 3:
        key.append(0)
    return tuple(key)

for head in head_to_page:
    key_to_head[gen_key(head)] = head

In [10]:
offset = -12

def dfs_add_label(node, level):
    if level > 2:
        return

    global parent
    curr_parent = parent

    list_node = list(node)
    list_node[level] += 1

    while tuple(list_node) in key_to_head:
        head = key_to_head[tuple(list_node)]
        pg_no = head_to_page[head] + offset
        parent = writer.add_outline_item(head, pg_no, parent=curr_parent)
        dfs_add_label(tuple(list_node), level+1)
        list_node[level] += 1

parent = None

for ch in range(1,10):
    temp = (ch, 0, 0)
    head = key_to_head[temp]
    pg_no = head_to_page[head] + offset
    parent = writer.add_outline_item(head, pg_no)
    dfs_add_label(temp, 1)

with open('Pattern Classification Duda (With Outline).pdf', 'wb') as output:
    writer.write(output)